### get_schedule

In [1]:
from NBA import NBA
import threading
import time
import os
import pandas as pd

In [2]:
def get_schedule(year, month):
    
    sch_url = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"

    sch_col_attrs = {"column_attrs": ['thead', 'th'], "shift": 0, "offset": ['0', None]}
    sch_row_attrs = {"row_attrs": ['tbody', 'tr'], "row_offset": ['0', None]}
    sch_data_attrs = {"data_attrs": ['th', 'td'], "data_offset": 0}
    
    sch = NBA(url = sch_url, 
              table_id = 'schedule',
              column_schema = sch_col_attrs, 
              row_schema = sch_row_attrs, 
              data_schema = sch_data_attrs, 
              name_change = {6:"Boxscore", 7: "Overtime"},
              use_links=[6],
              filter_rows = {"Date": "Playoffs"})   

    sch.df['season'] = year
    
    if len(sch.df) > 0:
        sch.df.to_csv(f"data/schedule/{year}_{month}.csv", index=False)

In [5]:
def collapse_files(folder, save):
    
    final = pd.DataFrame()
    
    for file in os.listdir(folder):
        final = pd.concat([final, pd.read_csv(f"{folder}/{file}")])
        os.remove(f"{folder}/{file}")
    
    final.to_csv(f"{folder}/{save}")

In [3]:
if __name__ == '__main__':
    
    if not os.path.exists("data/schedule"):
        os.makedirs("data/schedule")
        
    years = list(range(2001, 2023))
    
    months = ["october", "november", "december", "january", "february", "march", "april", "may", "june"]
    
    months_2020 = ["october-2019", "november", "december", "january", "february", "march", "july",
                   "august", "september", "october-2020"]
    
    months_2021 = ["december", "january", "february", "march", "april", "may", "june", "july"]
    
    for year in years:
        print(f"Getting {year} NBA data.")
        
        if year == 2021:
            loop_months = months_2021
        elif year == 2020:
            loop_monts = months_2020
        else:
            loop_months = months
        
        for month in months:
            clean_month = month.replace('-', '')
            exec(f"t_{clean_month} = threading.Thread(target=get_schedule, args=(year, month,))")
            exec(f"t_{clean_month}.start()")
        for month in months:
            exec(f"t_{clean_month}.join()")
    
    print('Collapsing Files') 
    collapse_files("data/schedule", "nba_schedule.csv", index=False)

Getting 2001 NBA data.
Getting 2002 NBA data.
Getting 2003 NBA data.
Getting 2004 NBA data.
Getting 2005 NBA data.
Getting 2006 NBA data.
Getting 2007 NBA data.
Getting 2008 NBA data.
Getting 2009 NBA data.
Getting 2010 NBA data.
Getting 2011 NBA data.
Getting 2012 NBA data.
Getting 2013 NBA data.
Getting 2014 NBA data.
Getting 2015 NBA data.
Getting 2016 NBA data.
Getting 2017 NBA data.
Getting 2018 NBA data.
Getting 2019 NBA data.
Getting 2020 NBA data.
Getting 2021 NBA data.
Getting 2022 NBA data.


AttributeError: type object 'NBA' has no attribute 'collapse_files'

### test

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("data/schedule/nba_schedule.csv")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27920 entries, 0 to 27919
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       27920 non-null  int64 
 1   Date             27920 non-null  object
 2   Start (ET)       27920 non-null  object
 3   Visitor/Neutral  27920 non-null  object
 4   PTS              27920 non-null  int64 
 5   Home/Neutral     27920 non-null  object
 6   PTS.1            27920 non-null  int64 
 7   Boxscore         27920 non-null  object
 8   Overtime         1694 non-null   object
 9   Attend.          27905 non-null  object
 10  Arena            27920 non-null  object
 11  Notes            49 non-null     object
 12  season           27920 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 2.8+ MB
